In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
void PrepareModel(ThermalModelBase * &model, ThermalParticleSystem *parts, 
                  const string& ensemble, const string& width_scheme) {
    if (ensemble == "GCE") {
        model = new ThermalModelIdeal(parts);

    }
    else {
        model = new ThermalModelCanonical(parts);

        // For SCE treat B & Q grand-canonically
        if (ensemble == "SCE") {
            static_cast<ThermalModelCanonical*>(model)->ConserveBaryonCharge(false);
            static_cast<ThermalModelCanonical*>(model)->ConserveElectricCharge(false);
        }
    }

    ThermalModelParameters params;
    // Chemical potentials are fixed to zero
    params.muB = 0.0;
    params.muQ = 0.0;
    params.muS = 0.0;
    params.muC = 0.0;

    // Initial temperature value in fits
    params.T = 0.155;

    // Quantum numbers are zero
    params.B = params.Q = params.S = params.C = 0;

    model->SetParameters(params);

    // Quantum statistics only for pions
    model->SetStatistics(true);
    for (int i = 0; i < model->TPS()->Particles().size(); ++i) {
        ThermalParticle &part = model->TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }

    if (ensemble != "GCE") {
        static_cast<ThermalModelCanonical*>(model)->CalculateQuantumNumbersRange();
    }

    // Set resonance widths scheme

    if (width_scheme == "ZeroWidth") {
        model->SetUseWidth(ThermalParticle::ZeroWidth);
    }
    else if (width_scheme == "BWTwoGamma") {
        model->SetUseWidth(ThermalParticle::BWTwoGamma);
    }
    else {
        model->SetUseWidth(ThermalParticle::eBW);
    }

    model->FillChemicalPotentials();
}

In [5]:
// Particle list
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

In [6]:
vector<int> pdgs1, pdgs2;
vector<string> names1, names2;

names1.push_back("K");
names2.push_back("pi");
pdgs1.push_back(321);
pdgs2.push_back(211);

names1.push_back("Xi");
names2.push_back("pi");
pdgs1.push_back(3312);
pdgs2.push_back(211);

names1.push_back("phi");
names2.push_back("pi");
pdgs1.push_back(333);
pdgs2.push_back(211);

names1.push_back("p");
names2.push_back("pi");
pdgs1.push_back(2212);
pdgs2.push_back(211);

names1.push_back("Omega");
names2.push_back("pi");
pdgs1.push_back(3334);
pdgs2.push_back(211);

names1.push_back("La");
names2.push_back("pi");
pdgs1.push_back(3122);
pdgs2.push_back(211);

names1.push_back("K*");
names2.push_back("pi");
pdgs1.push_back(323);
pdgs2.push_back(211);

names1.push_back("omega(782)");
names2.push_back("pi");
pdgs1.push_back(223);
pdgs2.push_back(211);

names1.push_back("K*0");
names2.push_back("K-");
pdgs1.push_back(313);
pdgs2.push_back(-321);

In [7]:
double TchVsNch(double dNchdEta) {
    return 0.176 - 0.0026 * log(dNchdEta);
}

In [8]:
double gammaSVsNch(double dNchdEta) {
    return 1. - 0.25 * exp(-dNchdEta / 59.);
}

In [9]:
double dVdyVsNch(double dNchdEta) {
    return 2.4 * dNchdEta;
}

In [10]:
vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);

In [11]:
void PerformScan(ThermalModelBase *model, 
                   double k,
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "dNch/deta";
        *out << setw(15) << "Tch[MeV]";
        *out << setw(15) << "dVdy[fm3]";
        *out << setw(15) << "Vc[fm3]";
        *out << setw(15) << "gammaS";

        for (int i = 0; i < names1.size(); ++i) {
            *out << setw(15) << (names1[i] + "/" + names2[i]) + "";
        }

        *out << endl;
    }
    
    double Nchmin = 3.;
    double Nchmax = 2000.;
    int iters = 100;
    double logNchmin = log10(Nchmin), logNchmax = log10(Nchmax);
    double dlogNch = (logNchmax - logNchmin) / (iters - 1);
    
    for (double logNch = logNchmin; logNch <= logNchmax + 1.e-3; logNch += dlogNch) {
        double Nch = pow(10., logNch);
        
        double Tch = TchVsNch(Nch);
        double gammaS = gammaSVsNch(Nch);
        double dVdy = dVdyVsNch(Nch);
        double Vc = k * dVdy;

        model->SetTemperature(Tch);
        model->SetGammaS(gammaS);
        model->SetVolume(dVdy);
        model->SetCanonicalVolume(Vc);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << Nch;
            *out << setw(15) << Tch;
            *out << setw(15) << dVdy;
            *out << setw(15) << Vc;
            *out << setw(15) << gammaS;

            for (int i = 0; i < names1.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgs1[i], 1) / model->GetDensity(pdgs2[i], 1);
            }

            *out << endl;
        }
    }
    
    fout.close();
}

In [12]:
ThermalModelBase *model;
PrepareModel(model, &parts, "CE", "eBW");

In [13]:
PerformScan(model,3.,"out/gsCSM.dNchdEtaDep.Vc.eq.3dVdy.dat");

      dNch/deta       Tch[MeV]      dVdy[fm3]        Vc[fm3]         gammaS           K/pi          Xi/pi         phi/pi           p/pi       Omega/pi          La/pi          K*/pi  omega(782)/pi         K*0/K-
              3       0.173144            7.2           21.6       0.762394       0.132441     0.00233768      0.0190051      0.0582325    0.000182838      0.0260147      0.0445997       0.133656       0.331795
        3.20365       0.172973        7.68877        23.0663       0.763213       0.133727     0.00246297      0.0188597        0.05955    0.000199068      0.0267606      0.0450422       0.132655       0.331871
        3.42113       0.172802        8.21072        24.6322       0.764084       0.134899     0.00258544      0.0187303      0.0607645    0.000215773      0.0274536      0.0454372       0.131731        0.33188
        3.65337       0.172631         8.7681        26.3043       0.765011       0.135967     0.00270441      0.0186158       0.061876    0.000232849      

In [14]:
ThermalModelBase *modelgce;
PrepareModel(modelgce, &parts, "GCE", "eBW");

In [15]:
PerformScan(modelgce,3.,"out/gsCSM.dNchdEtaDep.GCE.dat");

      dNch/deta       Tch[MeV]      dVdy[fm3]        Vc[fm3]         gammaS           K/pi          Xi/pi         phi/pi           p/pi       Omega/pi          La/pi          K*/pi  omega(782)/pi         K*0/K-
              3       0.173144            7.2           21.6       0.762394       0.145716      0.0048261      0.0169256      0.0787515    0.000714976      0.0369392      0.0494687       0.121637       0.334632
        3.20365       0.172973        7.68877        23.0663       0.763213       0.145831     0.00482024      0.0169382      0.0785453    0.000713934      0.0368664      0.0494622        0.12155       0.334318
        3.42113       0.172802        8.21072        24.6322       0.764084       0.145954     0.00481492      0.0169527      0.0783383     0.00071302      0.0367955      0.0494581       0.121461       0.334002
        3.65337       0.172631         8.7681        26.3043       0.765011       0.146085     0.00481018      0.0169692      0.0781306    0.000712239      